# Referenzdatensatz erstellen

In [49]:
# Besonderheit HASOC 2020 Daten: Zeilenumbrüche innerhalb der Tweets
# --> notwendig, die Dateien erst so aufzubereiten, dass nur ein Tweet pro Zeile steht

# HASOC 2020 Train: "..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new.txt"
# HASOC 2020 Test: "..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new.txt"

def rem_white(filename):
    with open(filename, mode="r", encoding="utf-16") as f:
        content = f.readlines()
        content = content[1:] # Erklärungszeile ignorieren
        newcont = []
        for i in range(len(content)):
            # Fall 1: Zeile ist komplett
            if len(content[i].split("\t")) == 5:
                newcont.append(content[i])
            # Fall 2: Zeile ist nicht komplett
            else:
                # letzter Teil einer Zeile erreicht
                if "hasoc_2020_de_" in content[i]:
                    comp_line += content[i]
                    comp_line = comp_line.replace("\n"," ")
                    comp_line += "\n"
                    newcont.append(comp_line)
                    comp_line = "" # Zeilenakkumulator zurücksetzen
                # erster Teil einer Zeile 
                elif content[i].startswith("11") :
                    comp_line = content[i]
                # mittlerer Teil einer Zeile, manchmal nur \n
                else: comp_line += content[i]
    nwfilename = filename[:len(filename)-4] + "_formatted" + ".txt"
    with open(nwfilename, mode="w", encoding="utf-8") as outfile:
        for line in newcont:
            outfile.write(line)
    return True

# Bereits formatiert, nicht nochmals durchführen
# rem_white("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new.txt")
# rem_white("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new.txt")

In [88]:
# Besonderheit des Covid19-Abusive-Datensatzes: noch nicht unterteilt in Trainings- und Testdaten

from sklearn.model_selection import train_test_split

with open("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset.csv", mode="r", encoding="utf-8") as infile:
    content = infile.readlines()
    explanation = content[0].split("\t")
    explanation = "\t".join([explanation[0],explanation[1],explanation[2],explanation[3]]) +"\n"
    content = content[1:]
    sep_content = [entry.split("\t") for entry in content]
    tweets = [(entry[0], entry[1], entry[2]) for entry in sep_content]
    labels = [entry[3] for entry in sep_content]

    # Separate into train/test
    # insg. 4960 Tweets, laut Paper unterteilt in Trainings-Set mit 3485 Tweets (70%), Validations-Set mit 735, und Test-Set mit 740 Tweets
    # hier entsprechend der Split 70/30, Validation-Daten werden also im Zweifelsfall von dem Testset abgespalten
    traintweets, testtweets, trainlabels, testlabels = train_test_split(tweets, labels, test_size=0.3, train_size=0.7, random_state=4, stratify=labels)

    # Reassemble into filewritable content
    train = []
    test = []
    for i, tweet in enumerate(traintweets):
        train.append("\t".join([tweet[0],tweet[1],tweet[2],trainlabels[i]]))
    for j, tweet in enumerate(testtweets):
        test.append("\t".join([tweet[0],tweet[1],tweet[2],testlabels[j]]))

with open("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_train.txt", mode="w", encoding="utf-8") as trainout:
    trainout.write(explanation)
    for train_point in train:
        trainout.write(train_point+"\n")

with open("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_test.txt", mode="w", encoding="utf-8") as testout:
    testout.write(explanation)
    for test_point in test:
        testout.write(test_point+"\n")

In [24]:
# Besonderheit des GermEval2019-Datensatzes: abgeschnittene erste Zeilen in den Testdaten
# --> notwendig, die Dateien aufzubereiten

# fehlende Buchstaben am Anfang von Tweets: abgeschnitten von GermEval2019 intern beim Übergang der Daten von
# "Testdata_Subtask12" zu "GoldLabelSubtask12"; Zeilennr. sind die gleichen
# z.B. für Zeile 341:
# "enschen, die etwas auf eBay-Kleinanzeigen verticken, wohnen entweder in einer dickichten Gartenlaube oder im obersten Stock Altbau - Hinterhaus. #issso	OTHER	OTHER"


# Testdaten: mit den Originaltweets aus der Datei "germeval2019_Testdata_Subtask12.txt" (dort ohne Labels) speichern

with open("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2.txt", mode="r", encoding="utf-8") as in_test:
    cont_test = in_test.readlines()
    sep_cont = [line.strip().split("\t") for line in cont_test]

# Tweets ohne abgeschnitte Anfänge einlesen
with open("..\Korpora\GermEval-2019-Data\germeval2019_Testdata_Subtask12.txt", mode="r", encoding="utf-8") as in_tweets:
    tweets = in_tweets.readlines()
    tweets = [tweet.strip() for tweet in tweets]

# Tweets mit den Labels zusammenführen
tweets_replaced = [(tweets[i],sep_cont[i][1],sep_cont[i][2]) for i in range(len(sep_cont))]

with open("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2_ersetzt.txt", mode="w", encoding="utf-8") as out_test:
    for line in tweets_replaced:
        out_test.write("\t".join(line)+"\n")

In [10]:
import re

def convert_to_refcorp(filename, corp_id, mod):
    """
    GermEval-Daten, HASOC-Daten und die Daten des Covid19-Abusive-Datensatzes in ein einheitliches Format zu übertragen.
    Input: Datei mit Tabstopp-getrennten Werten (ob txt, csv oder tsv), Korpus-ID, train/test-Information
    Output: Liste von Tupeln à (Referenzkorpus-ID, Tweet, Label1, Label2)
            - ReferenzkorpusID - setzt sich zusammen aus der Korpus-ID,
                                 md_id = "11", falls es um Trainingsdaten (mod=train), "22", falls es um Testdaten (mod=test) geht
                                 und der Zeilennummer in der Ursprungsdatei;
                                 also z.B.: "01220034" - für einen Tweet der Zeile 34, aus den Testdaten des GermEval2018-Datensatzes
            - Tweet            - String des Tweets, URLs sind mit der generischen Twitter-URL "https://t.co" ersetzt
            - Label1           -
            - Label2           -
    """
    newcorp = []
    with open(filename, mode="r", encoding="utf-8") as f:
        text = f.readlines()
        
        # erste Zeile ignorieren bei Covid19 ("05") und HASOC2019 ("03")
        if corp_id == "05" or corp_id == "03": text = text[1:]

        # Bestimmen, welche Formatierungsfunktion genutzt wird
        if corp_id == "01" or corp_id == "02": form_func = format_germeval
        elif corp_id == "03" or corp_id == "04": form_func = format_hasoc
        else: form_func = format_covidabusive

        url_pattern = re.compile('https:\/\/.*?(?: |$)')

        for num, entry in enumerate(text):
            entry = entry.strip()
            tag1, tag2 = "NOT", "NOT"

            tweet, tag1, tag2 = form_func(entry, tag1, tag2)

            # URLs mit generischer Twitter-URL ersetzen
            tweet = url_pattern.sub("https://t.co ", tweet)
            tweet = tweet.strip()

            # Tweet von HTML-Resten entfernen und Emoji-Codierung mit Emojis ersetzen
            tweet = clean_tweet(tweet)

            # ID erstellen
            if mod == "train": md_id = "11"
            elif mod =="test": md_id = "22"
            id_num = f'{num+1:04d}'
            tweet_id = str(corp_id) + str(md_id) + str(id_num)
            
            # der neuen Sammlung hinzufügen
            newcorp.append((tweet_id, tweet, tag1, tag2))
    return newcorp

def format_germeval(entry, tag1, tag2):
    tweet, label1, label2 = entry.split("\t")
    if label1 == "OFFENSE": tag1 = "NEG"
    if label2 == "INSULT": tag2 = "INSOFF"
    elif label2 == "PROFANITY": tag2 = "PRFN"
    elif label2 == "ABUSE": tag2 = "HATE"
    tweet = tweet.replace("|LBR|", " ")
    return tweet, tag1, tag2

def format_covidabusive(entry, tag1, tag2):
    sep = entry.split("\t")
    tweet, l1 = sep[1], sep[3]
    tag2 = "NAN"
    if l1 == "abusive": tag1 = "NEG"
    return tweet, tag1, tag2

def format_hasoc(entry, tag1, tag2):
    sep = entry.split("\t")
    tweet, l1, l2 = sep[1], sep[2], sep[3]
    if l1 == "HOF": tag1 = "NEG"
    if l2 == "HATE": tag2 = "HATE"
    elif l2 == "OFFN": tag2 = "INSOFF"
    elif l2 == "PRFN": tag2 = "PRFN"
    return tweet, tag1, tag2



# - 
# - html-Reste: "&amp;" anstatt "&", "\''" ansatt '"' und "&gt;" anstatt ">" usw. >> in die utf-8-Formatierung überführen
def clean_tweet(tweet):
    """Emojis finden und ersetzen (gefunden: https://stackoverflow.com/questions/67507017/replace-unicode-code-point-with-actual-character-using-regex)
        und HTML-Reste entfernen.
    """
    cleaned = tweet
    # Emojis, die als Text, z.B. "<U+0001F60A>", gespeichert sind: als utf-8 formatieren
    cleaned = re.sub(r'<U\+([A-F0-9]+)>', lambda x: chr(int(x.group(1), 16)), cleaned)
    cleaned = re.sub(r"&lt;" , "<", cleaned)	 
    cleaned = re.sub(r"&gt;" , ">", cleaned)    
    cleaned = re.sub(r"&amp;" , "&", cleaned)
    cleaned = re.sub(r'\"', '"', cleaned)
    cleaned = re.sub(r'\""', '"', cleaned)
    return cleaned

In [13]:
# GermEval2018
# Train: "..\Korpora\GermEval-2018-Data-master\germeval2018.training.txt"
# Test: "..\Korpora\GermEval-2018-Data-master\germeval2018.test.txt"
germeval2018train_converted = convert_to_refcorp("..\Korpora\GermEval-2018-Data-master\germeval2018.training.txt", "01", "train")
germeval2018test_converted = convert_to_refcorp("..\Korpora\GermEval-2018-Data-master\germeval2018.test.txt", "01", "test")

# GermEval2019
# Train: "..\Korpora\GermEval-2019-Data\germeval2019.training_subtask1_2_korrigiert.txt"
# Test: "..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2_ersetzt.txt"
germeval2019train_converted = convert_to_refcorp("..\Korpora\GermEval-2019-Data\germeval2019.training_subtask1_2_korrigiert.txt", "02", "train")
germeval2019test_converted = convert_to_refcorp("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2_ersetzt.txt", "02", "test")

In [14]:
# HASOC 2019
# Train: Korpora\german_dataset_hasoc2019\german_dataset\german_dataset.tsv
# Test: Korpora\german_dataset_hasoc2019\german_dataset\hasoc_de_test_gold.tsv
hasoc2019train_converted = convert_to_refcorp("..\Korpora\german_dataset_hasoc2019\german_dataset\german_dataset.tsv", "03", "train")
hasoc2019test_converted = convert_to_refcorp("..\Korpora\german_dataset_hasoc2019\german_dataset\hasoc_de_test_gold.tsv", "03", "test")

# HASOC 2020
# Train: Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new_formatted.txt
# Test: Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new_formatted.txt
hasoc2020train_converted = convert_to_refcorp("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new_formatted.txt", "04", "train")
hasoc2020test_converted = convert_to_refcorp("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new_formatted.txt", "04", "test")

In [15]:
# Covid19 Abusive

# Korpus ursprünglich: "..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset.csv"
# Train (neu): "..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_train.txt"
# Test (neu): "..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_test.txt"

covidabusivetrain_converted = convert_to_refcorp("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_train.txt", "05", "train")
covidabusivetest_converted = convert_to_refcorp("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_test.txt", "05", "test")

In [16]:
# Referenzdatensatz zusammenstellen
# Bereits ausgeführt, nicht noch einmal ausführen
# zweimal ausgeführt (korrigierte Formatierungen und IDs) --> Shuffle hat die Reihenfolge verändert
# (allerdings irrelevant, da alle Tweets IDs haben)

refcorp_train = germeval2018train_converted + germeval2019train_converted + hasoc2019train_converted + hasoc2020train_converted + covidabusivetrain_converted
refcorp_test = germeval2018test_converted + germeval2019test_converted + hasoc2019test_converted + hasoc2020test_converted + covidabusivetest_converted

# refcorp_gsmt = refcorp_train + refcorp_test # insg. 28596 Tweets
# refcorp_gsmt_HATE = [entry for entry in refcorp_gsmt if entry[3] == "HATE"] # insg. 3060 Tweets

import random

random.shuffle(refcorp_train)
random.shuffle(refcorp_test)

#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train.txt", mode="w", encoding="utf-8") as reftrainout:
#    reftrainout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")
#    for reftweet in refcorp_train:
#        reftrainout.write("\t".join(reftweet)+"\n")
    
#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test.txt", mode="w", encoding="utf-8") as reftestout:
#    reftestout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")
#    for reftweet in refcorp_test:
#        reftestout.write("\t".join(reftweet)+"\n")

In [160]:
# Ausschnittdatensätze (Train, Test) erstellen, in dem nur die Einträge mit dem feinen Label "HATE" vorkommen
# Bereits erstellt, nicht nochmals ausführen
# zweimal ausgeführt (korrigierte Formatierungen und IDs) --> Shuffle hat die Reihenfolge der Ausgangsdatei verändert
# (allerdings irrelevant, da alle Tweets IDs haben)


#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test.txt", mode="r", encoding="utf-8") as hatetrainin:
#    all_cont = hatetrainin.readlines()
#    all_cont = all_cont[1:]
#    sep_cont = [entry.strip().split("\t") for entry in all_cont]
#    hate = []
#    for tweet in sep_cont:
#        if tweet[3] == "HATE": hate.append(tweet)

#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE.txt", mode="w", encoding="utf-8") as hatetrainout:
#    for line in hate:
#        hatetrainout.write("\t".join(line)+"\n")


#### Train/Test - HATE Dateien des Referenzdatensatzes als JSON-Dateien speichern

In [162]:
#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE.txt", mode="r", encoding="utf-8") as intxt:
#    tws = intxt.readlines()
#    conts = [elem.strip("\n").split("\t") for elem in tws]
#    cont_dicts = [{"id":elem[0], "text":elem[1], "tag1":elem[2], "tag2":elem[3]} for elem in conts]

#import json

#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE.json", mode="w", encoding="utf-8") as jsonout:
#    prep_cont = json.dumps(cont_dicts)
#    jsonout.write(prep_cont)

## Annotation: Logikprüfung

In [88]:
def check_anno(datensatz):
    """Annotationslogik checken und Problemfälle sammeln"""
    probleme = []
    for entry in datensatz:
        if not anno_logik_check(entry["label"]): probleme.append(entry)
    return probleme


def anno_logik_check(labelset):
    """ Die Annotationslogik checken:
        Input: Labelmenge als Liste
        Output: True (falls alles korrekt) / False (falls irgendein Problem vorliegt)    
    """
    labelset = set(labelset)
    korrekt = True
    if len(labelset) == 0:
        korrekt = False
    elif (len(labelset) == 1) and ("KEINE" not in labelset):
        korrekt = False
    else:
        # 1. VVH-ALLG interne Logik
        if "VVH-ALLG" in labelset:
            # Mind. eine Art der Tathandlung ggb.
            if {"Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"} & labelset == set():
                korrekt = False
            # Mind. eine Gruppe genannt
            if {"Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
                    "Politische Einstellung", "Geschlecht", "Anderes Merkmal"} & labelset == set():
                korrekt = False
            # keine VVH-NS Labels
            if {"VVH-NS", "Billigen", "Verherrlichen", "Verharmlosen", "Leugnen", "Rechtfertigen"} & labelset != set():
                korrekt = False
            if "KEINE" in labelset: korrekt = False

        # 2. VVH-NS interne Logik
        if "VVH-NS" in labelset:
            # Mind. eine Art der Tathandlung ggb.
            if {"Billigen", "Verherrlichen", "Verharmlosen", "Leugnen", "Rechtfertigen"} & labelset == set(): 
                korrekt = False
            # keine VVH-ALLG Labels
            if {"VVH-ALLG", "Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"} & labelset != set():
                korrekt = False 
            if "KEINE" in labelset: korrekt = False
        
        # 3. Gruppenmerkmale - Logik (in beide Richtungen)

        nation = {"Türkischstämmige Deutsche", "Marokkaner:innen", "In Deutschland lebende Ausländer:innen",
                    "Asiat:innen", "Pol:innen", "Afrikaner:innen", "Syrer:innen", "Palästinenser:innen"}
        herkunft = {"POC", "Araber:in"}
        religion = {"Muslim:e/innen", "Juden/Jüdinnen", "Christ:innen"}
        polit = {"Die Grünen", "die SPD", "die Linke", "CDU/CSU", "AfD", "Nazis", "Islamist:innen", "Kommunist:innen",
                    "Zionist:innen", "NPD", "FDP", "FridaysForFuture", "Pegida", "Anarchist:in"}
        geschlecht = {"Trans/NB-Personen", "Frauen", "Männer"}
        andere = {"Flüchtlinge", "Asylbewerber:innen", "Sich illegal in Deutschland aufhaltende Personen", "Migrant:innen", "Vorbestrafte",
                    "Veganer", "Senior:innen", "Lesben, Schwule, Bi", "Kinder", "Jugendliche", "Polizist:innen", "Obdachlose",
                    "Richter:innen", "Analphabet:innen", "Soldat:innen", "Behinderte"}

        # Richtung 1: falls Gruppe vorhanden, korrektes Merkmal auch vorhanden
        for i in nation & labelset:
            if "Nationalität" not in labelset:
                korrekt = False
        for j in herkunft & labelset:
            if 'ethnische Herkunft / "Rasse"' not in labelset:
                korrekt = False
        for k in religion & labelset:
            if "Religion / Weltanschauung" not in labelset:
                korrekt = False
        for l in polit & labelset:
            if "Politische Einstellung" not in labelset:
                korrekt = False
        for m in geschlecht & labelset:
            if "Geschlecht" not in labelset:
                korrekt = False
        for n in andere & labelset:
            if "Anderes Merkmal" not in labelset:
                korrekt = False

        # Richtung 2: falls Merkmal vorhanden, auch eine passende Gruppe vorhanden
        if "Nationalität" in labelset:
            if nation & labelset == set():
                korrekt = False
        if 'ethnische Herkunft / "Rasse"' in labelset:
            if herkunft & labelset == set():
                korrekt = False
        if "Religion / Weltanschauung" in labelset:
            if religion & labelset == set():
                korrekt = False
        if "Politische Einstellung" in labelset:
            if polit & labelset == set():
                korrekt = False
        if "Geschlecht" in labelset:
            if geschlecht & labelset == set():
                korrekt = False
        if "Anderes Merkmal" in labelset:
            if andere & labelset == set():
                korrekt = False

    return korrekt

In [97]:
# Probleme in der Annotationslogik händisch durchgehen

import json

# Jeweils für die Trainingsdaten und die Testdaten
# Beispieleintrag der Annotations-Json-Datei:
# {"id": "01112520", "data": "@SteiblBarbara @Thomas_S_Wagner @RitaKratzert Weitaus schlimmer. Heute ist es nicht mehr Dummheit.  Ein ganzes Volk ist zu (m)wutlosen Zombies dressiert worden.", "label": ["KEINE"], "tag1": "NEG", "tag2": "HATE"}
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_anno_korrigiert.json", mode="r", encoding="utf-8") as trainf:
    train_anno = trainf.read()
    train_annotations = json.loads(train_anno)

probleme = check_anno(train_annotations)

[]


In [78]:
# Korrekturen für die Trainingsdaten
 
new01110262 = {'id': '01110262', 'label': ['KEINE', 'Flüchtlinge', 'Afrikaner:innen', 'Nationalität', 'Anderes Merkmal']}
new03113327 = {'id': '03113327', 'label': ['Angriff der Menschenwürde', 'VVH-ALLG', 'POC', 'ethnische Herkunft / "Rasse"', 'Kinder', 'Die Grünen', 'Politische Einstellung', 'Anderes Merkmal'] }
new01114326 = {'id': '01114326', 'label': ['Muslim:e/innen', 'Religion / Weltanschauung', 'Islamist:innen', 'Aufstachelung zu Hass', 'VVH-ALLG', 'Politische Einstellung']}
new01111028 = {'id': '01111028', 'label': ['KEINE', 'Araber:in', 'ethnische Herkunft / "Rasse"']}
new03113648 = {'id': '03113648', 'label': ['KEINE', 'Politische Einstellung', 'Nazis']}
new01112254 = {'id': '01112254', 'label': ['KEINE', 'Pol:innen', 'In Deutschland lebende Ausländer:innen', 'Nationalität']}
new02110710 = {'id': '02110710', 'label': ['KEINE', 'Araber:in', 'In Deutschland lebende Ausländer:innen', 'ethnische Herkunft / "Rasse"', 'POC', 'Afrikaner:innen', 'Nationalität']}
new04111621 = {'id': '04111621', 'label': ['KEINE']}
new03112728 = {'id': '03112728', 'label': ['KEINE']}

# Entspricht damit nicht mehr der ursprünglichen Logik; Entscheidung: Palästinenser:innen eher Nationalität als Polit. Einstellung
# >> alle entsprechenden Einträge umschreiben
new02113786 = {'id': '02113786', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}
new01113048 = {'id': '01113048', 'label': ['Palästinenser:innen', 'Nationalität', 'VVH-ALLG', 'Aufstachelung zu Hass'], 'tag1': 'NEG', 'tag2': 'HATE'}
new01110156 = {'id': '01110156', 'label': ['Palästinenser:innen', 'Nationalität', 'VVH-ALLG', 'Aufstachelung zu Hass'], 'tag1': 'NEG', 'tag2': 'HATE'}


In [81]:
# Korrekturen für die Testdaten

new01222281 = {'id': '01222281', 'label': ['Die Grünen', 'Türkischstämmige Deutsche','Nationalität', 'Politische Einstellung', 'KEINE']}
new02220290 = {'id': '02220290', 'label': ['KEINE', 'die SPD', 'Politische Einstellung']}
new01221146  = {'id': '01221146', 'label': ['KEINE', 'die SPD', 'Politische Einstellung']}
new01220517 = {'id': '01220517', 'label': ['KEINE', 'In Deutschland lebende Ausländer:innen', 'Pol:innen', 'Migrant:innen', 'Nationalität', 'Anderes Merkmal']}
new02221882 = {'id': '02221882', 'label': ['KEINE', 'Zionist:innen', 'Palästinenser:innen', 'Politische Einstellung', 'Nationalität']}
new01223120 = {'id': '01223120', 'label': ['KEINE']}
new01221025 = {'id': '01221025', 'label': ['Verharmlosen', 'VVH-NS']}
new02222036 = {'id': '02222036', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}
new01220250 = {'id': '01220250', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}
new02221762 = {'id': '02221762', 'label': ['KEINE', 'AfD', 'Politische Einstellung', 'Islamist:innen', 'Araber:in', 'In Deutschland lebende Ausländer:innen', 'Nationalität', 'ethnische Herkunft / "Rasse"']}
new01223162 = {'id': '01223162', 'label': ['KEINE', 'Islamist:innen', 'Politische Einstellung']}
new02221146 = {'id': '02221146', 'label': ['Nazis', 'Aufforderung zu Gewalt- oder Willkürmaßnahmen', 'VVH-ALLG', 'Politische Einstellung']}
new01220249 = {'id': '01220249', 'label': ['KEINE', 'In Deutschland lebende Ausländer:innen', 'Nationalität']}
new02221991 = {'id': '02221991', 'label': ['KEINE', 'Palästinenser:innen', 'Zionist:innen', 'Politische Einstellung', 'Nationalität']}
new02220270 = {'id': '02220270', 'label': ['KEINE', 'Araber:in', 'Afrikaner:innen', 'In Deutschland lebende Ausländer:innen', 'Nationalität', 'Religion / Weltanschauung', 'Muslim:e/innen', 'ethnische Herkunft / "Rasse"']}
new02220311 = {'id': '02220311', 'label': ['KEINE', 'Araber:in', 'In Deutschland lebende Ausländer:innen', 'Afrikaner:innen', 'Muslim:e/innen', 'Religion / Weltanschauung', 'Nationalität', 'ethnische Herkunft / "Rasse"']}
new01222167 = {'id': '01222167', 'label': ['KEINE', 'In Deutschland lebende Ausländer:innen', 'Islamist:innen', 'Die Grünen', 'Politische Einstellung', 'Asylbewerber:innen', 'Anderes Merkmal', 'Nationalität']}
new01222700 = {'id': '01222700', 'label': ['KEINE', 'Anderes Merkmal', 'Sich illegal in Deutschland aufhaltende Personen']}
new01222612 = {'id': '01222612', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}

In [95]:
# erkannte Fehler: Trainings- und Testannotationsdateien umschreiben, inkl. der neuen IDs (je +1)
# in neue Dateien

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_annotiert.json", mode="r", encoding="utf-8") as trainf:
    train_anno = trainf.read()
    train_annotations = json.loads(train_anno)

korrekturen_train = [new01110262, new03113327, new01114326, new01111028, new03113648, new01112254, new02110710, new04111621, new03112728, new02113786, new01113048, new01110156]
for i in train_annotations:
    for j in korrekturen_train:
        if i["id"] == j["id"]:
            i["label"] = j["label"]
    i["id"] = i["id"][:4] + f'{int(i["id"][4:])+1:04d}'


train_anno_k = json.dumps(train_annotations)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_anno_korrigiert.json", mode="w", encoding="utf-8") as trainf_k:
    trainf_k.write(train_anno_k)


with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE_annotiert.json", mode="r", encoding="utf-8") as testf:
    test_anno = testf.read()
    test_annotations = json.loads(test_anno)

korrekturen_test = [new01222281, new02220290, new01221146, new01220517, new02221882, new01223120, new01221025, new02222036, new01220250,
                    new02221762, new01223162, new02221146, new01220249, new02221991, new02220270, new02220311, new01222167, new01222700, new01222612]

for i in test_annotations:
    for j in korrekturen_test:
        if i["id"] == j["id"]:
            i["label"] = j["label"]
    i["id"] = i["id"][:4] + f'{int(i["id"][4:])+1:04d}'

test_anno_k = json.dumps(test_annotations)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE_anno_korrigiert.json", mode="w", encoding="utf-8") as testf_k:
    testf_k.write(test_anno_k)